# Capstone Project Flask UI

Pada capstone ini anda diminta untuk membangun sebuah Flask UI sederhana yang berisi beberapa tampilan plot dari skeleton yang sudah disediakan dan satu plot tambahan berdasarkan analisis anda.

File ini dapat dimanfaatkan untuk membantu anda dalam proses wrangling dan visualization. Apabila proses wrangling sudah tepat dan hasil visualisasi sudah sesuai dengan ketentuan, anda dapat memindahkan kembali kode program yang sudah anda lengkapi ke dalam file `app.py`.

## Create Virtual Enviroment 

a. Created virtual environment called "capstone-flask"

Hal pertama yang harus dilakukan adalah melakukan pengaturan environment conda. Untuk menyiapkan conda environment dan kernel, silahkan gunakan command berikut:

```
conda create -n <ENV_NAME> python=3.10
conda activate <ENV_NAME>

conda install ipykernel
python -m ipykernel install --user --name <ENV_NAME>
```

b. Install packages: pandas, flask, matplotlib, dan numpy

Seluruh dependecies telah di-export ke dalam file requirements.txt. Oleh karena itu untuk melakukan install packages, Anda dapat menggunakan perintah berikut:

```
pip install -r requirements.txt --user
```

## Data Preprocessing

**Import library**

In [ ]:
from flask import Flask, render_template
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import base64

**Load data**

Bacalah file `googleplaystore.csv` data dan simpan ke objek dataframe dengan nama playstore

In [ ]:
playstore = pd.read_csv('data/googleplaystore.csv')

**Data Cleansing** 

**_Task 1:_** Hapus data yang duplikat berdasarkan kolom App, dengan tetap keep data pertama (hint : gunakan parameter subset)

In [ ]:
playstore.drop_duplicates(subset = ['App'], inplace = True) 

In [ ]:
# Tambahan cell - check duplicate
playstore.duplicated().sum()

Bagian ini untuk menghapus row 10472 karena nilai data tersebut tidak tersimpan pada kolom yang benar

In [ ]:
playstore.drop([10472], inplace=True)

**_Task 2:_** Cek tipe data kolom Category. Jika masih tersimpan dengan format tipe data yang salah, ubah ke tipe data yang sesuai


In [ ]:
# Tambahan cell
playstore.dtypes

In [ ]:
playstore.Category = playstore['Category'].astype('category')

**_Task 3:_** Pada kolom Installs Buang tanda koma(,) dan tanda tambah(+) kemudian ubah tipe data menjadi integer

In [ ]:
playstore.Installs = playstore['Installs'].apply(lambda x: x.replace(',', ''))
playstore.Installs = playstore['Installs'].apply(lambda x: x.replace('+', ''))

playstore.Installs = playstore['Installs'].astype('int64')

In [ ]:
# Tambahan cell
playstore.dtypes

In [ ]:
# Tambahan cell
playstore.head(3)

Bagian ini untuk merapikan kolom Size, Anda tidak perlu mengubah apapun di bagian ini

In [ ]:
playstore['Size'].replace('Varies with device', np.nan, inplace = True ) 
playstore.Size = (playstore.Size.replace(r'[kM]+$', '', regex=True).astype(float) * \
             playstore.Size.str.extract(r'[\d\.]+([kM]+)', expand=False)
            .fillna(1)
            .replace(['k','M'], [10**3, 10**6]).astype(int))
playstore['Size'].fillna(playstore.groupby('Category')['Size'].transform('mean'),inplace = True)


**_Task 4:_** Pada kolom Price, buang karakater $ pada nilai Price lalu ubah tipe datanya menjadi float

In [ ]:
playstore['Price'] = playstore['Price'].apply(lambda x: x.replace('$', ''))
playstore['Price'] = playstore['Price'].astype('float64')

In [ ]:
# Tambahan cell
playstore.dtypes

In [ ]:
# Tambahan cell
playstore.head(3)

Ubah tipe data Reviews, Size, Installs ke dalam tipe data integer

In [ ]:
##code here
list_int = ['Reviews', 'Size', 'Installs']
playstore[list_int] = playstore[list_int].astype('int64')

In [ ]:
# Tambahan cell
playstore.dtypes

## Data Wrangling

In [ ]:
df2 = playstore.copy()

**_Task 5:_** Dataframe top_category dibuat untuk menyimpan frekuensi aplikasi untuk setiap Category. 
Gunakan crosstab untuk menghitung frekuensi aplikasi di setiap category kemudian gunakan `Jumlah`
sebagai nama kolom dan urutkan nilai frekuensi dari nilai yang paling banyak. Terakhir reset index dari dataframe top_category.

In [ ]:
top_category = pd.crosstab(
    index = df2['Category'],
    columns = 'Jumlah'
).sort_values(by = 'Jumlah', ascending = False).reset_index()

In [ ]:
top_category.head()

**_Task 6:_** Ini adalah bagian untuk melengkapi konten value box 
most category mengambil nama category paling banyak mengacu pada dataframe `top_category`
total mengambil frekuensi/jumlah category paling banyak mengacu pada dataframe `top_category`

In [ ]:
most_categories = top_category['Category'][0]
total = top_category['Jumlah'][0]

`rev_table` adalah tabel yang berisi 10 aplikasi yang paling banyak direview oleh pengguna.
Silahkan melakukan agregasi data yang tepat menggunakan `groupby` untuk menampilkan 10 aplikasi yang diurutkan berdasarkan jumlah Review pengguna. 

Tabel yang ditampilkan terdiri dari 4 kolom yaitu nama Category, nama App, total Reviews, dan rata-rata Rating.
Pindahkan kode wrangling yang disimpan dalam variable `rev_table` pada `blank code` yang telah di chaining dengan kode `.to_html`.

In [ ]:
rev_table = df2.groupby(['Category', 'App']).agg({
    'Reviews' : 'sum', 
    'Rating' : 'mean'
}).sort_values('Reviews', ascending = False).head(10).reset_index()

Apabila menuliskan kode program yang tepat maka hasil wrangling adalah sebagai berikut :

In [ ]:
rev_table

## Data Visualization

**Membuat Bar Plot**

**_Task 7:_** 
Lengkapi tahap agregasi untuk membuat dataframe yang mengelompokkan aplikasi berdasarkan Category.
Buatlah bar plot dimana axis x adalah nama Category dan axis y adalah jumlah aplikasi pada setiap kategori, kemudian urutkan dari jumlah terbanyak

In [ ]:
cat_order = df2.groupby(['Category']).agg({
    'App' : 'count'
}).rename({'Category':'Total'}, axis=1).sort_values('App', ascending = False).head()

X = cat_order.reset_index()['Category']
Y = cat_order.reset_index()['App']
my_colors = ['r','g','b','k','y','m','c']
# bagian ini digunakan untuk membuat kanvas/figure
fig = plt.figure(figsize=(8,3),dpi=300)
fig.add_subplot()
# bagian ini digunakan untuk membuat bar plot
plt.barh(X, Y, color=my_colors)
# bagian ini digunakan untuk menyimpan plot dalam format image.png
plt.savefig('cat_order.png',bbox_inches="tight") 

**Membuat Scatter Plot**

**_Task 8:_** Buatlah scatter plot untuk menampilkan hubungan dan persebaran apalikasi dilihat dari Review vs Rating.
Ukuran scatter menggambarkan berapa banyak pengguna yang telah menginstall aplikasi 
    

In [ ]:
X = df2['Reviews'].values # axis x
Y = df2['Rating'].values # axis y
area = playstore['Installs'].values/10000000 # ukuran besar/kecilnya lingkaran scatter plot
fig = plt.figure(figsize=(5,5))
fig.add_subplot()
# isi nama method untuk scatter plot, variabel x, dan variabel y
plt.scatter(x=X, y=Y, s=area, alpha = 0.3)
plt.xlabel('Reviews')
plt.ylabel('Rating')
plt.savefig('rev_rat.png', bbox_inches='tight')

**Membuat Histogram Size Distribution**

**_Task 9:_** Buatlah sebuah histogram yang menggambarkan distribusi Size aplikasi dalam satuan Mb(Megabytes). Histogram yang terbentuk terbagi menjadi 100 bins

In [ ]:
X=(playstore['Size']/1000000).values
fig = plt.figure(figsize=(5,5))
fig.add_subplot()
plt.hist(X, bins=100, density=True,  alpha=0.75)
plt.xlabel('Size')
plt.ylabel('Frequency')
plt.savefig('hist_size.png',bbox_inches="tight")

**_Task 10:_** Buatlah plot berdasarkan hasil analisis anda. Plot yang dibuat tidak diperbolehkan sama dengan plot yang sudah dicontohkan.

In [ ]:
# Aggregasi Data untuk Subset Free App vs Paid App (Alt)

# Data apps playstore type Free saja
list_apps_free = playstore[(playstore['Type'] == 'Free')]

# Frekuensi applikasi setiap kategori khusus Apps type Free saja
top_category_free = pd.crosstab(
    index = list_apps_free['Category'],
    columns = 'Jumlah Frekuensi'
).sort_values(by = 'Jumlah Frekuensi', ascending = False)

# Data apps playstore type Paid saja
list_apps_paid = playstore[(playstore['Type'] == 'Paid')]

# Frekuensi applikasi setiap kategori khusus Apps type Paid saja
top_category_paid = pd.crosstab(
    index = list_apps_paid['Category'],
    columns = 'Jumlah Frekuensi'
).sort_values(by = 'Jumlah Frekuensi', ascending = False)

X1 = top_category_free.head(5).reset_index()['Category']
Y1 = top_category_free.head(5).reset_index()['Jumlah Frekuensi']

X2 = top_category_paid.head(5).reset_index()['Category']
Y2 = top_category_paid.head(5).reset_index()['Jumlah Frekuensi']

# Stacking subplots vertically
fig = plt.figure(figsize=(8,10),dpi=600)
fig, axs = plt.subplots(2)
fig.suptitle('Top 5 Most Frequent Category', fontsize=16)

axs[0].barh(X1, Y1, color = 'b')
# Show top values
axs[0].invert_yaxis()
axs[0].set_title('FREE APPS')

axs[1].barh(X2, Y2, color = 'm')
# Show top values
axs[1].invert_yaxis()
# Add Plot Title
axs[1].set_title('PAID APPS')

for ax in axs.flat:
    ax.set(xlabel='Jumlah Frekuensi', ylabel='Category')
    
fig.tight_layout()
plt.savefig('top_5_freq_cat_free_vs_paid.png')
